In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(14)
tf.config.threading.set_intra_op_parallelism_threads(14)

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

import numpy as np
import matplotlib.pyplot as plt

from pddlvm.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0
from pddlvm.VEModel import VE
from pddlvm.NeuralNet import NeuralN
from pddlvm.poisson1DPDEState import PDE_State
tf.random.set_seed(42)
import time
print(tffloat(0.))
tf.config.list_physical_devices()

prefix = 'NonLinHeatPINNPDDLVM'
dirname = '{}/'.format(prefix)
os.makedirs(os.path.dirname(dirname), exist_ok=True)

In [ ]:
dimZ      = 2
dimDomain = 2
dimU      = 1

In [ ]:
#2D grid
nx = 100
nt = 200
piTF = tf.constant(np.pi, dtype=tf.float32)
xi = tf.linspace(0., 2*piTF, nx)
ti = tf.linspace(0., 2., nt)

xmGrid, tmGrid = np.meshgrid(xi, ti)
xmGridBody, tmGridBody = np.meshgrid(xi[1:-1], ti[1:])
nxBody  = xi[1:-1].shape[0]
ntBody  = ti[1:].shape[0]
print((nxBody, ntBody))
print(nxBody * ntBody)


xmBody = tf.reshape(xmGridBody, [-1, 1])
tmBody = tf.reshape(tmGridBody, [-1, 1])

xm     = tf.reshape(xmGrid, [-1, 1])
tm     = tf.reshape(tmGrid, [-1, 1])

x = tf.reshape(xi, [-1,1])
t = tf.reshape(ti, [-1,1])

xbl = tf.repeat(x[0, 0], nt)[:, None]
xbr = tf.repeat(x[-1,0], nt)[:, None]
t0  = tf.repeat(t[0, 0], nx)[:, None]

xtbl = tf.concat([xbl, ti[:, None]], 1)
xtbr = tf.concat([xbr, ti[:, None]], 1)
xt0  = tf.concat([xi[:, None], t0], 1)

allICBC = tf.concat([xtbl, xtbr, xt0], 0)

print(xmGridBody.shape)
print(allICBC.shape)


In [ ]:
nh     = 100
U_XTK  = NeuralN('U_of_XTK', 
                np.array([  dimZ + dimDomain,
                            nh, nh, nh, nh,
                            dimU])
                )

K_U  = NeuralN('K_of_U',  np.array([  1, 50, dimZ]) )
# New definition of NeuralN for K_U; patched object outside class as conv not yet supported in class
K_U.NN = tf.keras.Sequential(
    [   tf.keras.layers.Reshape((nxBody,ntBody, 1), input_shape=(1, nxBody*ntBody)),
        tf.keras.layers.Conv2D(
            filters=8, kernel_size=4, strides=(2, 2), activation='swish'),
        tf.keras.layers.Conv2D(
            filters=16, kernel_size=4, strides=(2, 2), activation='swish'),
        tf.keras.layers.Conv2D(
            filters=32, kernel_size=4, strides=(2, 2), activation='swish'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(500),
        tf.keras.layers.Dense(4),
    ]
    )
K_U.NN.compile( )
K_U.NN.summary( )
#New definition of MAP for K_U; patched object outside class as conv not yet supported in class
from types import MethodType
def Map(self, varIn):    
    varIn  = self.transf_forward(varIn)
    mapped = self.NN( varIn[None,...] )
    mean, logvar = mapped[:,:self.layout[-1]], mapped[:,self.layout[-1]:]
    tf.debugging.assert_all_finite(mean, 'mean not finite')
    tf.debugging.assert_all_finite(logvar, 'logvar not finite')
    logvar = self.dBoundwalpha_log( logvar )
    mean   = tf.cast(mean, floatf)
    logvar = tf.cast(logvar, floatf)
    return mean, logvar
K_U.Map = MethodType(Map, K_U)

trainableVars = [U_XTK.NN.trainable_variables, K_U.NN.trainable_variables]
epsilon_r     = tf.Variable(tf.constant(1e-2), trainable=False)
# Z = gamma, kappa
LatentPriorZ      = tfp.distributions.Uniform(low=tffloat([1., 1.]), high=tffloat([5., 5.]) )

In [ ]:
def reparameterize( mean, logvar):
    eps = tf.random.normal(shape=[mean.shape[0], mean.shape[1]], dtype=floatf)
    z = eps * tf.exp(logvar * .5) + mean
    tf.debugging.assert_all_finite(z, 'z reparam not finite')
    return z    
    
log2pi = tf.math.log( tf.constant(2* np.pi, dtype=tf.float32))
def log_normal_pdf(sample, mean, logvar, raxis=1):
    return tf.reduce_sum( -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)


def mu_of_u( U, kappa ):
    # thermal conductivity as funciton of t
    return (U * kappa**2 + 1) / kappa

def residualFunction( z_i ):
    with tf.GradientTape(watch_accessed_variables=False) as t1:
        t1.watch(xmBody)
        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch(xmBody)
            t2.watch(tmBody)
            mean_alpha, logvar_alpha = U_XTK.Map( [xmBody, tmBody, tf.tile(z_i, [xmBody.shape[0], 1])] )
            us = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])
        grad_x_Us = tf.reshape(t2.gradient(us, xmBody), [-1,1] )
        grad_t_Us = tf.reshape(t2.gradient(us, tmBody), [-1,1] )
        muU       = mu_of_u( us, tf.squeeze(z_i[0, 1]) )
        muUGradUX = muU * grad_x_Us
    gradMuUGradUX = tf.reshape(t1.gradient(muUGradUX, xmBody), [-1,1] )
    residualBody = - grad_t_Us + 1./tf.squeeze(z_i[0, 0]) * gradMuUGradUX

    mean_alphaICBD, logvar_alphaICBD = U_XTK.Map( [allICBC, tf.tile(z_i, [allICBC.shape[0], 1])] )
    usICBD = tf.reshape(reparameterize(mean_alphaICBD, logvar_alphaICBD), [-1,1])
    uxl          = usICBD[:nt,     0] 
    uxr          = usICBD[nt:2*nt, 0] 
    uxt0         = usICBD[2*nt: ,  0] 
    u0, bcLeft, bcRight = tf.math.sin(xi) + 1., 1. , 1.
    residualICBC = tf.reshape(tf.concat([ ( uxt0 - u0 ), ( uxl - bcLeft ), ( uxr - bcRight ) ], 0), [-1,1])

    residual = tf.concat([residualBody, residualICBC], 0)

    logP_alpha_u_Z  = log_normal_pdf( us , mean_alpha, logvar_alpha, raxis=0)

    return us, residual, logP_alpha_u_Z


def ELBO( ):
    
    z_i = tf.reshape( LatentPriorZ.sample( ) , [-1, dimZ])

    us, residual, logP_alpha_u_Z = residualFunction( z_i )

    logVarEpsilonVectBody = tf.repeat(2*tf.math.log(epsilon_r), tf.reduce_prod(xmBody.shape) )[:, None]
    logVarEpsilonVectBCIC = tf.repeat(2*tf.math.log(epsilon_r/10.), allICBC.shape[0] )[:, None]
    logVarEpsilonVect     = tf.concat([logVarEpsilonVectBody , logVarEpsilonVectBCIC ], 0)

    logP_r   = log_normal_pdf( residual, tf.constant(0., shape=residual.shape), logVarEpsilonVect, raxis=0 )

    mean_beta, logvar_beta = K_U.Map( tf.reshape(us, [1,-1]) )

    logP_beta_Z_u          = log_normal_pdf( z_i, mean_beta, logvar_beta, raxis=1 ) #this is correct dimension

    elbo = tf.reduce_mean( logP_r + logP_beta_Z_u -  logP_alpha_u_Z )

    return elbo

print(ELBO())



In [ ]:
@tf.function
def train_step_prior(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = - ELBO( )
    for vars in trainableVars:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))
    return loss

num_iterations   = tfint(500_000)

epsilon_r.assign( tf.constant(1e-2) )

intervalELBOsave = 1000

optimizer = tf.keras.optimizers.Adam( 1e-3 )

elboAll     = []
startTraining = time.time()
for iteration in tf.range(0, num_iterations ):

    start_time = time.time()
    loss = train_step_prior(optimizer, iteration)

    end_time = time.time()

    if iteration % intervalELBOsave == 0:
        elboAll.append(-loss)
        print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
           .format(iteration, -loss, end_time - start_time))

endTraining = time.time()


In [ ]:
plt.plot(np.array(elboAll))
plt.ylim(-1e6, 1e6)
plt.show()
np.savetxt(dirname+'ELBO.dat', np.array(elboAll))

In [ ]:
z_i = tf.reshape( LatentPriorZ.sample( ) , [-1, dimZ])

z_i = tf.constant([[5., 5.]])

# for z_i in z:
mean_alpha, logvar_alpha = U_XTK.Map( [xm, tm, tf.tile(z_i, [xm.shape[0], 1])] )
stddev_alpha = tf.exp(logvar_alpha*0.5) * 1

allI = np.array(range(nt))[::10]
colors = plt.cm.viridis(np.linspace(0,1,allI.shape[0]))
fig, ax = plt.subplots()
ic = 0
c = np.arange(1, allI.shape[0] + 1)
cmap = plt.cm.get_cmap('viridis', allI.shape[0])

dummie_cax = ax.scatter(c, t[::10, 0], c=t[::10, 0], cmap=cmap)
ax.cla()
for i in allI:
    im = ax.plot(xm[i*nx:(i+1)*nx, 0], mean_alpha[i*nx:(i+1)*nx], c=colors[ic])
    ax.fill_between( tf.squeeze(xm[i*nx:(i+1)*nx, 0]),  tf.squeeze(mean_alpha[i*nx:(i+1)*nx] + 2* stddev_alpha[i*nx:(i+1)*nx]), 
                     tf.squeeze(mean_alpha[i*nx:(i+1)*nx] - 2* stddev_alpha[i*nx:(i+1)*nx]), color=colors[ic], alpha=0.5 )
    ic+=1

fig.colorbar(dummie_cax, label=r'$t$')
ax.set_ylabel(r'$u$')
ax.set_xlabel(r'$x$')
ax.set_title(r'$u(x,t) \pm 2\sigma(x,t)$')
plt.show()

SAVE = dirname

fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(5, 2.5))
im1 = ax.contourf(tmGrid, xmGrid, tf.reshape(mean_alpha, [nt, nx]), 100)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$x$')
fig.colorbar(im1, ax=ax)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(5, 2.5))
im2 = ax.contourf(tmGrid, xmGrid, tf.reshape(2*tf.exp(logvar_alpha*0.5), [nt, nx]), 100)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$x$')
fig.colorbar(im2, ax=ax)
plt.tight_layout()
plt.show()

_, finalRes, _ = residualFunction( z_i )
print("finalRes = ", tf.reduce_sum(finalRes ** 2)/finalRes.shape[0] )

In [ ]:
saveModelWeights = False
loadModelWeights = False

if saveModelWeights:
    U_XTK.NN.save_weights(dirname+'U_of_K_weights.h5')
    K_U.NN.save_weights(dirname+'K_of_U_weights.h5')
    U_XTK.NN.save(dirname+'U_of_K_weights.model')
    K_U.NN.save(dirname+'K_of_U_weights.model')
    print('SAVED MODEL WEIGHTS')
    

if loadModelWeights:

    U_XTK.NN = tf.keras.models.load_model(dirname+'U_of_K_weights.model')
    K_U.NN   = tf.keras.models.load_model(dirname+'K_of_U_weights.model')
    U_XTK.NN.compile()
    K_U.NN.compile()
    # Check its architecture
    U_XTK.NN.summary()
    K_U.NN.summary()
    print('LOADED MODEL WEIGHTS')


In [ ]:
nGamma = 100
nKappa = 100

gamma = tf.linspace(0.1, 12., nGamma)
kappa = tf.linspace(0.1, 12., nKappa)

gammaGrid, kappaGrid = np.meshgrid(gamma, kappa)

gammam     = tf.reshape(gammaGrid, [-1, 1])
kappam     = tf.reshape(kappaGrid, [-1, 1])
Params     = tf.concat([gammam, kappam], 1)

paramSpaceResidual = []
paramSpaceStddev   = [] 
betastddev         = []
zResidual          = [] 
betastddevMEAN     = []

for i in range(nGamma * nKappa):

    z_i = Params[i, :][None, :]

    mean_alpha, logvar_alpha = U_XTK.Map( [xmBody, tmBody, tf.tile(z_i, [xmBody.shape[0], 1])] )
    _, pdeResidual, _ = residualFunction( z_i )
    pdeResidual = tf.reduce_sum(pdeResidual ** 2)/pdeResidual.shape[0]
    paramSpaceResidual.append( pdeResidual )
    print(pdeResidual)

    stdAVG      = tf.reduce_mean( 2*tf.exp( logvar_alpha * 0.5 ) )
    paramSpaceStddev.append( stdAVG )

    mean_beta, logvar_beta = K_U.Map( tf.reshape(mean_alpha, [1,-1]) )
    print("mean_beta, logvar_beta = ",mean_beta, logvar_beta)
    NSEZ = ( tf.linalg.norm( z_i - mean_beta) / tf.linalg.norm( z_i ) ) ** 2
    zResidual.append( NSEZ )
    betastdAVG      = tf.reduce_mean( 2*tf.exp( logvar_beta[0, 1] * 0.5 ) )
    betastddev.append( betastdAVG )
    betastddevMEAN.append( mean_beta[0,0] )

paramSpaceResidualNP = np.log( np.array( paramSpaceResidual ).reshape(-1,1) )
paramSpaceStddevNP   = np.log( np.array( paramSpaceStddev ).reshape(-1,1) )

zResidualNP          = np.log( np.array( zResidual ).reshape(-1,1) )
betastddevNP         = np.log( np.array( betastddev ).reshape(-1,1) )
betastddevMEANNP     = np.array( betastddevMEAN ).reshape(-1,1) 


In [ ]:

parameterBox = np.array([[1,1], [1, 5], [5, 5], [5, 1], [1, 1]])
SAVE = dirname
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, figsize=(6, 5))
im1 = ax.contourf(gammaGrid, kappaGrid, tf.reshape(paramSpaceResidualNP, [nGamma, nKappa]), 200)
ax.plot(parameterBox[:,0], parameterBox[:,1], c='w')
ax.set_xlabel(r'$\gamma$')
ax.set_ylabel(r'$\kappa$')
fig.colorbar(im1, ax=ax)
plt.tight_layout( )
plt.savefig( '{}gammaKappaGridPDEResidual-1210.png'.format(SAVE), dpi=300 )
plt.show()


fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, figsize=(6, 5))
im2 = ax.contourf(gammaGrid, kappaGrid, tf.reshape(paramSpaceStddevNP,  [nGamma, nKappa]), 200)
ax.plot(parameterBox[:,0], parameterBox[:,1], c='w')
ax.set_xlabel(r'$\gamma$')
ax.set_ylabel(r'$\kappa$')
plt.tight_layout( )
fig.colorbar(im2, ax=ax)
plt.savefig( '{}gammaKappaGridUstddev-1210.png'.format(SAVE), dpi=300 )
plt.show( )

plt.show()

In [ ]:
nGamma = 20
nKappa = 20

gamma = tf.linspace(0.5, 7., nGamma)
kappa = tf.linspace(0.5, 7., nKappa)

gammaGrid, kappaGrid = np.meshgrid(gamma, kappa)

gammam     = tf.reshape(gammaGrid, [-1, 1])
kappam     = tf.reshape(kappaGrid, [-1, 1])
Params     = tf.concat([gammam, kappam], 1)

paramSpaceResidual = []
paramSpaceStddev   = [] 
betastddev         = []
zResidual          = [] 

for i in range(nGamma * nKappa):

    z_i = Params[i, :][None, :]

    mean_alpha, logvar_alpha = U_XTK.Map( [xmBody, tmBody, tf.tile(z_i, [xmBody.shape[0], 1])] )
    _, pdeResidual, _ = residualFunction( z_i )
    pdeResidual = tf.reduce_sum(pdeResidual ** 2)/pdeResidual.shape[0]
    paramSpaceResidual.append( pdeResidual )
    print(pdeResidual)

    stdAVG      = tf.reduce_mean( 2*tf.exp( logvar_alpha * 0.5 ) )
    paramSpaceStddev.append( stdAVG )

    mean_beta, logvar_beta = K_U.Map( tf.reshape(mean_alpha, [1,-1]) )
    NSEZ = ( tf.linalg.norm( z_i - mean_beta) / tf.linalg.norm( z_i ) ) ** 2
    zResidual.append( NSEZ )
    betastdAVG      = tf.reduce_mean( 2*tf.exp( logvar_beta * 0.5 ) )
    betastddev.append( betastdAVG )

paramSpaceResidualNP = np.log( np.array( paramSpaceResidual ).reshape(-1,1) )
paramSpaceStddevNP   = np.log( np.array( paramSpaceStddev ).reshape(-1,1) )
zResidualNP          = np.log( np.array( zResidual ).reshape(-1,1) )
betastddevNP         = np.log( np.array( betastddev ).reshape(-1,1) )

fig, ax = plt.subplots(nrows=1, ncols=2, sharex=False, figsize=(20, 6))

parameterBox = np.array([[1,1], [1, 5], [5, 5], [5, 1], [1, 1]])

im3 = ax[0].contourf(gammaGrid, kappaGrid, tf.reshape(zResidualNP,  [nGamma, nKappa]), 200)
ax[0].set_title(r'$\log NSE_{\mathbf{z}}(\gamma, \kappa)$')
ax[0].plot(parameterBox[:,0], parameterBox[:,1], c='w')
ax[0].set_xlabel(r'$\gamma$')
ax[0].set_ylabel(r'$\kappa$')

im4 = ax[1].contourf(gammaGrid, kappaGrid, tf.reshape(betastddevNP,  [nGamma, nKappa]), 200)
ax[1].set_title(r'$\log 2\bar{\sigma}_{\beta}(\gamma, \kappa)$')
ax[1].plot(parameterBox[:,0], parameterBox[:,1], c='k')
ax[1].set_xlabel(r'$\gamma$')
ax[1].set_ylabel(r'$\kappa$')

fig.colorbar(im3, ax=ax[0])
fig.colorbar(im4, ax=ax[1])
plt.tight_layout()

SAVE = dirname+'inRangePriors'

extent = ax[0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())

extent = ax[1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
plt.show()


In [ ]:
# PURE PINN SECTION

layout = np.array([4,100,100,100,100, 1])
PINN = tf.keras.Sequential( )
activation = tf.keras.activations.swish
initializer = tf.keras.initializers.HeNormal()

for i in range(1, layout.shape[0] - 1):
    if i == 1: 
        print('first layer')
        PINN.add(tf.keras.layers.Dense(layout[i], input_dim = layout[0], activation = activation, kernel_initializer=initializer))
    else:
        print('inner layer ', i)
        PINN.add(tf.keras.layers.Dense(layout[i], activation = activation, kernel_initializer=initializer))
print('last layer')
PINN.add(tf.keras.layers.Dense(  layout[-1]  ))
PINN.summary()

In [ ]:
nGammaPinn = 50
nKappaPinn = 50

gammaPinn = tf.linspace(1., 5., nGammaPinn)
kappaPinn = tf.linspace(1., 5., nKappaPinn)

gammaGridPinn, kappaGridPinn = np.meshgrid(gammaPinn, kappaPinn)

gammamPinn     = tf.reshape(gammaGridPinn, [-1, 1])
kappamPinn     = tf.reshape(kappaGridPinn, [-1, 1])
ParamsPinn     = tf.concat([gammamPinn, kappamPinn], 1)


num_iterations = 500_000

optimizer = tf.keras.optimizers.Adam( 1e-3 )

def modelNNPINN(x, t, z):
    return PINN( tf.concat([x, t, z], 1) )
def modelPDDLVMMeanForward(x, t, z):
    mean_alpha, logvar_alpha = U_XTK.Map( [x, t, z] )
    return mean_alpha
modelFUNC = modelNNPINN

def residualFunctionPINN( z_i ):
    with tf.GradientTape(watch_accessed_variables=False) as t1:
        t1.watch(xmBody)
        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch(xmBody)
            t2.watch(tmBody)
            U = modelFUNC( xmBody, tmBody, tf.tile(z_i, [xmBody.shape[0], 1]) )
        grad_x_Us = tf.reshape(t2.gradient(U, xmBody), [-1,1] )
        grad_t_Us = tf.reshape(t2.gradient(U, tmBody), [-1,1] )
        muU       = mu_of_u( U, tf.squeeze(z_i[0, 1]) )
        muUGradUX = muU * grad_x_Us
    gradMuUGradUX = tf.reshape(t1.gradient(muUGradUX, xmBody), [-1,1] )
    residualBody = - grad_t_Us + 1./tf.squeeze(z_i[0, 0]) * gradMuUGradUX
    residualBodySummed = tf.reduce_sum(residualBody ** 2) / residualBody.shape[0]

    UCICBD = modelFUNC( allICBC[:, 0][:, None], allICBC[:, 1][:, None], tf.tile(z_i, [allICBC.shape[0], 1]) ) 
    uxl          = UCICBD[:nt,     0] 
    uxr          = UCICBD[nt:2*nt, 0] 
    uxt0         = UCICBD[2*nt: ,  0] 
    u0, bcLeft, bcRight = tf.math.sin(xi) + 1., 1. , 1.

    residualBC   = tf.reduce_sum( ( uxl - bcLeft ) ** 2 + ( uxr - bcRight ) ** 2 ) / (uxl.shape[0] + uxr.shape[0] )
    residualIC   = tf.reduce_sum( ( uxt0 - u0 ) ** 2 ) / (uxt0.shape[0] )
    residual     = residualBodySummed + residualBC + residualIC

    return residual

@tf.function
def train_step_pinn(optimizer, z_i):
    with tf.GradientTape() as tape:
        loss = residualFunctionPINN( z_i )
        gradients = tape.gradient(loss, PINN.trainable_variables)
        optimizer.apply_gradients(zip(gradients, PINN.trainable_variables))
    return loss

LOSS = []
for i in tf.range(num_iterations):
    index = i % ParamsPinn.shape[0]
    loss = train_step_pinn( optimizer, ParamsPinn[index, :][None, :] )
    LOSS.append( tf.squeeze(loss) )
    if i % 1000 == 0:
        print("iter ", i, " ", loss)

In [ ]:
saveModelWeights = False
loadModelWeights = False

if saveModelWeights:
    PINN.compile()
    PINN.save(dirname+'PINN_weights.model')
    print('SAVED MODEL WEIGHTS')
    

if loadModelWeights:
    PINN = tf.keras.models.load_model(dirname+'PINN_weights.model')    
    PINN.compile()
    print('LOADED MODEL WEIGHTS')

In [ ]:
nGamma = 100
nKappa = 100

gamma = tf.linspace(0.1, 12., nGamma)
kappa = tf.linspace(0.1, 12., nKappa)

gammaGrid, kappaGrid = np.meshgrid(gamma, kappa)

gammam     = tf.reshape(gammaGrid, [-1, 1])
kappam     = tf.reshape(kappaGrid, [-1, 1])
Params     = tf.concat([gammam, kappam], 1)

paramSpaceResidualPINN = []
paramSpaceResidualMEANPDDLVM = []


modelFUNC = modelNNPINN
for i in range(nGamma * nKappa):
    z_i = Params[i, :][None, :]
    paramSpaceResidualPINN.append( residualFunctionPINN( z_i ))

modelFUNC = modelPDDLVMMeanForward
for i in range(nGamma * nKappa):
    z_i = Params[i, :][None, :]
    paramSpaceResidualMEANPDDLVM.append( residualFunctionPINN( z_i ))


paramSpaceResidualPINNNP = np.log( np.array( paramSpaceResidualPINN ).reshape(-1,1) )
paramSpaceResidualMEANPDDLVMNP   = np.log( np.array( paramSpaceResidualMEANPDDLVM ).reshape(-1,1) )


In [ ]:

parameterBox = np.array([[1,1], [1, 5], [5, 5], [5, 1], [1, 1]])

fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, figsize=(6, 5))
im1 = ax.contourf(gammaGrid, kappaGrid, tf.reshape(paramSpaceResidualPINNNP, [nGamma, nKappa]), 200)
ax.plot(parameterBox[:,0], parameterBox[:,1], c='w')
ax.set_xlabel(r'$\gamma$')
ax.set_ylabel(r'$\kappa$')
fig.colorbar(im1, ax=ax)
plt.tight_layout()
plt.savefig( '{}paramSpaceResidualPINNNP2.png'.format(dirname) )
plt.show()

fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, figsize=(6, 5))
im2 = ax.contourf(gammaGrid, kappaGrid, tf.reshape(paramSpaceResidualMEANPDDLVMNP,  [nGamma, nKappa]), 200)
ax.plot(parameterBox[:,0], parameterBox[:,1], c='w')
ax.set_xlabel(r'$\gamma$')
ax.set_ylabel(r'$\kappa$')
plt.tight_layout()
fig.colorbar(im2, ax=ax)
plt.savefig( '{}paramSpaceResidualMEANPDDLVMNP2.png'.format(dirname) )
plt.show()


In [ ]:
nGamma = 100
nKappa = 100

gamma = tf.linspace(1., 5., nGamma)
kappa = tf.linspace(1., 5., nKappa)

gammaGrid, kappaGrid = np.meshgrid(gamma, kappa)

gammam     = tf.reshape(gammaGrid, [-1, 1])
kappam     = tf.reshape(kappaGrid, [-1, 1])
Params     = tf.concat([gammam, kappam], 1)

paramSpaceResidualPINN = []
paramSpaceResidualMEANPDDLVM = []


modelFUNC = modelNNPINN
for i in range(nGamma * nKappa):
    z_i = Params[i, :][None, :]
    paramSpaceResidualPINN.append( residualFunctionPINN( z_i ))

modelFUNC = modelPDDLVMMeanForward
for i in range(nGamma * nKappa):
    z_i = Params[i, :][None, :]
    paramSpaceResidualMEANPDDLVM.append( residualFunctionPINN( z_i ))


paramSpaceResidualPINNNP = np.log( np.array( paramSpaceResidualPINN ).reshape(-1,1) )
paramSpaceResidualMEANPDDLVMNP   = np.log( np.array( paramSpaceResidualMEANPDDLVM ).reshape(-1,1) )

print( 'meanInSquarePINN = ', np.mean(np.array(paramSpaceResidualPINN)) )
print( 'meanInSquarePDDLVM = ', np.mean(np.array(paramSpaceResidualMEANPDDLVM)) )